In [3]:
import os
from os import listdir
from os.path import isfile, join
import sys
import pandas as pd
import random
import numpy as np
import pickle as pkl
from operator import add 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression, Lasso

from utils import *

import TCA

In [4]:
def prepare_data(project, metric):
    data_path = '../data/700/merged_data_original/' + project + '.csv'
    data_df = pd.read_csv(data_path)
    data_df.rename(columns = {'Unnamed: 0':'id'},inplace = True)

    for col in ['id', 'commit_hash', 'release']:
        if col in data_df.columns:
            data_df = data_df.drop([col], axis = 1)
    data_df = data_df.dropna()
    y = data_df.Bugs
    X = data_df.drop(['Bugs'],axis = 1)
    if metric == 'process':
        X = X[['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr']]
    elif metric == 'product':
        X = X.drop(['file_la', 'file_ld', 'file_lt', 'file_age', 'file_ddev',
    'file_nuc', 'own', 'minor', 'file_ndev', 'file_ncomm', 'file_adev',
    'file_nadev', 'file_avg_nddev', 'file_avg_nadev', 'file_avg_ncomm',
    'file_ns', 'file_exp', 'file_sexp', 'file_rexp', 'file_nd', 'file_sctr'],axis = 1)
    else:
        X = X
    df = X
    df['Bugs'] = y
    return df

In [22]:
all_data = pd.DataFrame()
for i in [1,2,4,5,6,7,8]:
    with open('results/TCA/process/tca_'+ str(i) +'.pkl', 'rb') as f:
        data = pkl.load(f)
    all_data = pd.concat([all_data,data], axis = 0)
    all_data.reset_index(inplace = True, drop = True)

In [23]:
all_data

,source,target,recell,pf,g,f,pci_20
0,SoLoader,ShapeOfView,0.43,0.47,0.45,0.43,0.78
1,SoLoader,swipe-button,0.57,0.14,0.61,0.53,0.44
2,SoLoader,SmoothProgressBar,0.34,0.08,0.39,0.46,0.59
3,SoLoader,JustAuth,0.30,0.26,0.34,0.34,0.89
4,SoLoader,Android-ReactiveLocation,0.22,0.16,0.26,0.30,0.62
...,...,...,...,...,...,...,...
53265,elasticsearch-jdbc,Shizuku,0.90,0.48,0.79,0.72,0.63
53266,elasticsearch-jdbc,BannerViewPager,0.87,0.69,0.64,0.72,0.73
53267,elasticsearch-jdbc,typescript-generator,0.95,0.18,0.92,0.90,0.57
53268,elasticsearch-jdbc,pojobuilder,0.98,0.39,0.87,0.92,0.66


In [24]:
all_data.to_pickle('results/TCA/process/tca.pkl')

In [26]:
prediction_df = pd.read_csv('results/mixed_data/level_2/fold_1/predicted_source_process.csv')
prediction_df = prediction_df.drop(['Unnamed: 0'], axis = 1)
prediction_df = prediction_df.sort_values(['trg'])

In [30]:
src_projects = []
source_p = set()
for trg in prediction_df.trg.unique():
    sub_df = prediction_df[prediction_df['trg'] == trg]
    sub_df.reset_index(inplace = True, drop = True)
    print(trg,sub_df.iloc[sub_df.f.idxmax()].src, sub_df.f.max())
    src_projects.append([trg,sub_df.iloc[sub_df.f.idxmax()].src,sub_df.iloc[sub_df.pci_20.idxmax()].src])
    source_p.add(sub_df.iloc[sub_df.pci_20.idxmax()].src)

Android-DirectoryChooser intellij-elixir 0.63
Android-Rich-text-Editor intellij-elixir 0.63
AndroidPerformanceMonitor intellij-elixir 0.64
AndroidTraining intellij-elixir 0.57
Calligraphy intellij-elixir 0.64
CustomBottomSheetBehavior intellij-elixir 0.64
DaggerMock intellij-elixir 0.64
DataflowTemplates intellij-elixir 0.64
FinestWebView-Android intellij-elixir 0.64
FizzBuzzEnterpriseEdition intellij-elixir 0.64
FloatingActionButton intellij-elixir 0.64
HtmlSpanner intellij-elixir 0.64
ListItemView intellij-elixir 0.64
MultiImagePicker intellij-elixir 0.64
MultiViewAdapter intellij-elixir 0.63
OpenNoteScanner intellij-elixir 0.64
Paper intellij-elixir 0.64
QRCodeReaderView intellij-elixir 0.64
Quicksql intellij-elixir 0.64
RxBluetooth intellij-elixir 0.63
RxJavaFX intellij-elixir 0.64
SocialSdkLibrary intellij-elixir 0.64
UrlImageViewHelper intellij-elixir 0.64
WeatherLib intellij-elixir 0.64
X2C intellij-elixir 0.64
airline intellij-elixir 0.64
android-contentprovider-generator intel

In [31]:
source_p

{'SoLoader'}

In [32]:
f1 = []
for project_pairs in src_projects:
    tgt_name, recall, pf, g, f, pci_20 = TCA.tca_plus_test(project_pairs[1:],[project_pairs[0]])
    print(tgt_name, recall, pf, g, f, pci_20)
    f1.append(f)

Android-DirectoryChooser 0.46 0.28 0.5 0.49 0.71
Android-Rich-text-Editor 0.37 0.35 0.4 0.26 0.81
AndroidPerformanceMonitor 0.37 0.3 0.41 0.49 0.65
AndroidTraining 0.59 0.48 0.57 0.38 0.44
Calligraphy 0.62 0.62 0.55 0.46 0.79
CustomBottomSheetBehavior 0.69 0.32 0.69 0.58 0.58
DaggerMock 0.35 0.25 0.39 0.34 0.78
DataflowTemplates 0.4 0.73 0.36 0.22 0.78


KeyboardInterrupt: 

In [20]:
f1

[0.65,
 0.08,
 0.58,
 0.49,
 0.26,
 0.11,
 0.18,
 0.51,
 0.44,
 0.03,
 0.32,
 0.83,
 0.37,
 0.39,
 0.39,
 0.29,
 0.41,
 0.26,
 0.24,
 0.44,
 0.18,
 0.42,
 0.34,
 0.22,
 0.42,
 0.33,
 0.59,
 0.5,
 0.02,
 0.49,
 0.29,
 0.49,
 0.62,
 0.33,
 0.17,
 0.21,
 0.39,
 0.41,
 0.32,
 0.34,
 0.44,
 0.0,
 0.29,
 0.44,
 0.47,
 0.02,
 0.83,
 0.45,
 0.41,
 0.6,
 0.37,
 0.5,
 0.17,
 0.65,
 0.39,
 0.06,
 0.49,
 0.61,
 0.1,
 0.3,
 0.4,
 0.28,
 0.51,
 0.41,
 0.03,
 0.44,
 0.23,
 0.29,
 0.3,
 0.42,
 0.57,
 0.29,
 0.53,
 0.33,
 0.34,
 0.28,
 0.24]

In [ ]:
def _replaceitem(x):
    if x >= 0.5:
        return 0.5
    else:
        return 0.0

myList = [0,0.8,0.77]
a = [x / 2 for x in myList]
b = list(map(_replaceitem, myList))

In [ ]:
c = list(map(add, a, b))

In [ ]:
c

In [ ]:
def _replaceitem2(x):
    if x >= 0.5:
        return 1
    else:
        return 0

In [ ]:
z = list(map(_replaceitem2, c))
z

In [ ]:
data = pd.read_csv('data.csv')
data = data.drop('Unnamed: 0', axis = 1)

In [ ]:
train_y = data.f
train_X = data.drop('f', axis = 1)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR

In [ ]:
clf = SVR()
clf.fit(train_X,train_y)